In [ ]:
import os
import io
import contextlib

from dotenv import load_dotenv

load_dotenv()

BRIGHTDATA_SERP_API_KEY = os.getenv("BRIGHTDATA_SERP_API_KEY")
assert BRIGHTDATA_SERP_API_KEY is not None, "BRIGHTDATA_SERP_API_KEY is not set in the environment variables."



In [ ]:
%pip install langchain_brightdata

from langchain_brightdata import BrightDataSERP

serp_tool = BrightDataSERP(
    
    bright_data_api_key=BRIGHTDATA_SERP_API_KEY,
    search_engine="google",
    country="us",
    parse_results = True,    
    num_results=15,
    timeout=20,
    zone = "serp_api1"  # Optional: specify the zone if needed

    )




In [ ]:
import inspect
from langchain_brightdata import BrightDataSERP
print(inspect.signature(BrightDataSERP.__init__))


In [ ]:
buf = io.StringIO()
with contextlib.redirect_stdout(buf):
    result_data = serp_tool.invoke("Top 5 reddit communities for programming")
    # If the API returned an error string (like the "zone not found" message), handle it:
    if isinstance(result_data, str):
        print("Bright Data error:", result_data)
    else:
        # Pretty-print and iterate results
        import json
        print(json.dumps(result_data, indent=2))

        # Depending on BrightDataSERP, results are usually under "organic"
        for item in result_data.get("organic", []):
            print(f"Title:  {item.get('title')}")
            print(f"Link:   {item.get('link')}")
            print(f"Snippet:{item.get('snippet')}")
            print("-" * 40)

In [ ]:
import os, requests
from urllib.parse import quote_plus

api_token = os.getenv("BRIGHTDATA_SERP_API_KEY")
zone = "serp_api1"
q = "Top 5 reddit communities for programming"

encoded_q = quote_plus(q)

resp = requests.post(
    "https://api.brightdata.com/request",
    headers={
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json",
    },
    json={
        "zone": zone,
        "url": f"https://www.google.com/search?q={encoded_q}",
        "country": "us",
        "format": "json",   # required
        # "parse": True     # <-- remove this (not allowed)
    },
    timeout=30,
)

data = resp.json() if "application/json" in resp.headers.get("content-type","") else resp.text
print(type(data), "\n")
print(data)

############################
# data.keys() 
# data.get("organic")
# ###################################
# for result in data.get("organic", []):
#     print(f"Title: {result.get('title')}")
#     print(f"Link: {result.get('link')}")
#     print(f"Snippet: {result.get('snippet')}")
#     print("-" * 40)

In [ ]:
# import json
# data = json.dumps(result_data, indent=2)
# print(data)


In [ ]:
# # print(data)
# data.keys() 
# data.get("organic")

In [ ]:
# `resp_data` is the raw dict from requests.post(...).json()
resp_data = data  # whatever variable you used for resp.json()

# If you want to see it, pretty-print WITHOUT overwriting:
import json
print(json.dumps(resp_data, indent=2))

# Unwrap the payload
payload = resp_data.get("body", resp_data)  # some responses nest under "body"

# If body is a JSON string, parse it
if isinstance(payload, str):
    try:
        payload = json.loads(payload)
    except Exception:
        print("No parsed JSON results, raw body HTML below:")
        print(payload[:1000])  # preview
        payload = {}

# Now extract typical SERP fields
organic = []
if isinstance(payload, dict):
    # try common layouts
    if "organic" in payload and isinstance(payload["organic"], list):
        organic = payload["organic"]
    elif "results" in payload and isinstance(payload["results"], dict):
        organic = payload["results"].get("organic", [])
    elif "data" in payload and isinstance(payload["data"], dict):
        organic = payload["data"].get("organic", [])

# Display
for item in organic:
    if isinstance(item, dict):
        print(f"Title:   {item.get('title')}")
        print(f"Link:    {item.get('link') or item.get('url')}")
        print(f"Snippet: {item.get('snippet') or item.get('description')}")
        print("-" * 40)

if not organic:
    print("No 'organic' list found in payload structure.")
